In [58]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.express as px
import yfinance as yf

In [59]:
# SPDR ETFs
assets = [
    'XLC',  # Communication Services
    'XLY',  # Consumer Discretionary
    'XLP',  # Consumer Staples
    'XLE',  # Energy
    'XLF',  # Financials
    'XLV',  # Health Care
    'XLI',  # Industrials
    'XLB',  # Materials
    'XLRE', # Real Estate
    'XLK',  # Technology
    'XLU',  # Utilities
    'SPY',  # S&P 500
    'BIL'   # 1-3 Month Treasury Bill
]

# Download data from Yahoo Finance since 2018-06-19
start = '2018-06-19'
data = {}
for ticker in assets:
    data[ticker] = yf.download(ticker, start=start)

# Calculate returns and drop the first row
for ticker in assets:
    data[ticker]['Return'] = data[ticker]['Adj Close'].pct_change().iloc[1:]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [60]:
# Graph the cumulative returns
fig = px.line()
for ticker in assets:
    fig.add_scatter(x=data[ticker].index, y=(data[ticker]['Return']+1).cumprod(), name=ticker)
fig.update_layout(title='Cumulative Returns of SPDR ETFs',
                  xaxis_title='Date',
                  yaxis_title='Cumulative Return')
fig.show()

# Graph the returns
fig = px.line()
for ticker in assets:
    fig.add_scatter(x=data[ticker].index, y=data[ticker]['Return'], name=ticker)
fig.update_layout(title='Returns of SPDR ETFs',
                  xaxis_title='Date',
                  yaxis_title='Return')
fig.show()

In [61]:
def min_var_portfolio(m, sigma, mu_b):
    '''
    m: row vector of expected returns
    sigma: covariance matrix of returns
    mu_b: expected portfolio return
    
    Returns the weights and variance of the minimum variance portfolio
    '''
    m = np.matrix(m).T
    sigma = np.matrix(sigma)
    e = np.matrix(np.ones(len(m))).T
    sigma_inv = sigma.I
    # Calculate the minimum variance and market porfolio weights
    w_min_var = (sigma_inv * e) / (e.T * sigma_inv * e)
    w_mk = (sigma_inv * m) / (e.T * sigma_inv * m)
    # Calculate the return-constrained minimum variance portfolio
    v = w_mk - w_min_var
    alpha = ((mu_b - m.T * w_min_var) / (m.T * v))[0, 0]
    w = w_min_var + alpha * v
    return w, w.T * sigma * w

In [62]:
# Calculate the mean and covariance matrix of returns
returns = pd.DataFrame()
for ticker in assets:
    returns[ticker] = data[ticker]['Return']
m = returns.mean()
sigma = returns.cov()
w, var = min_var_portfolio(m, sigma, 0.0005)

# Graph the efficient frontier 
means = np.linspace(0.000025, 0.0001, 1000)
variances = np.vectorize(lambda mu: min_var_portfolio(m, sigma, mu)[1])(means)
fig = px.line(x=np.sqrt(variances), y=means)
fig.update_layout(title='Efficient Frontier of SPDR ETFs',
                  xaxis_title='Standard Deviation',
                  yaxis_title='Mean')
fig.update_xaxes(range=[0.000175, 0.0005])
fig.show()